# Optimización de Pesos de Evaluación (Versión Definitiva)

Restricciones:
1. **Granularidad**: Módulos divididos en Quices y Tareas.
2. **Regla de Quices**: Quices < 30% del módulo.
3. **Énfasis en Módulo 2**: $W_{M2} = 1.1 \times W_{M1}$ (M2 es un 10% mayor).
4. **Concepto Profesor**: Nota discreta (2.0, 2.5, ..., 5.0) basada en rendimiento previo, peso **<= 15%**.
5. **Reglas Globales**:
   - Examen Final >= 20% y **<= 40%**
   - Tarea Individual <= 20%
   - Zona Gris: Penalizamos el rango [2.90, 3.10].

## 1. Configuración Inicial

In [176]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

np.random.seed(42)

## 2. Generación de Escenarios (Distribuciones STEM)

Usamos generadores más realistas:
- Dist. Bimodal (para clases 'filtro').
- Dist. Sesgada Negativa (para tareas donde la mayoría aprueba).
- Dist. Normal (casos estándar).

In [177]:
def generar_distribucion_bimodal(n, low_center=1.5, high_center=4.5, ratio=0.5, sigma=0.6):
    n_high = int(n * ratio)
    n_low = n - n_high
    high_grades = np.random.normal(high_center, sigma, n_high)
    low_grades = np.random.normal(low_center, sigma, n_low)
    combined = np.concatenate([high_grades, low_grades])
    return np.clip(combined, 0, 5)

def generar_distribucion_negative_skew(n, mode=4.2, sigma=1.0):
    raw = mode - np.random.exponential(scale=0.8, size=n)
    return np.clip(raw, 0, 5)

def generar_datos_cientificos(n_estudiantes=2000):
    data = {}
    
    # MÓDULO 1: Fácil (Intro)
    data['M1_Q'] = generar_distribucion_negative_skew(n_estudiantes, mode=4.5)
    data['M1_T'] = generar_distribucion_negative_skew(n_estudiantes, mode=4.2)
    
    # MÓDULO 2: Filtro (Bimodal)
    data['M2_Q'] = generar_distribucion_bimodal(n_estudiantes, low_center=1.2, high_center=4.0, ratio=0.4)
    data['M2_T'] = generar_distribucion_bimodal(n_estudiantes, low_center=2.0, high_center=4.5, ratio=0.5) 
    
    # MÓDULO 3: Avanzado (Normal)
    data['M3_Q'] = np.clip(np.random.normal(3.0, 1.2, n_estudiantes), 0, 5)
    data['M3_T'] = np.clip(np.random.normal(3.5, 1.0, n_estudiantes), 0, 5)
    
    # EXAMEN FINAL
    data['Examen_Final'] = np.clip(np.random.normal(2.8, 1.1, n_estudiantes), 0, 5)
    
    df = pd.DataFrame(data)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df.round(2)

df_notas = generar_datos_cientificos(2000)
features = ['M1_Q', 'M1_T', 'M2_Q', 'M2_T', 'M3_Q', 'M3_T', 'Examen_Final']
display(df_notas.describe())
df_notas.head()

,M1_Q,M1_T,M2_Q,M2_T,M3_Q,M3_T,Examen_Final
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.00000
mean,3.699445,3.417365,2.308250,3.190665,3.008075,3.44423,2.80637
std,0.795124,0.760018,1.490038,1.340696,1.148524,0.92744,1.06758
min,0.000000,0.000000,0.000000,0.070000,0.000000,0.00000,0.00000
25%,3.390000,3.097500,1.070000,1.970000,2.230000,2.81000,2.10000
50%,3.930000,3.660000,1.735000,3.230000,3.010000,3.47000,2.80000
75%,4.280000,3.970000,3.792500,4.470000,3.850000,4.12000,3.53000
max,4.500000,4.200000,5.000000,5.000000,5.000000,5.00000,5.00000


,M1_Q,M1_T,M2_Q,M2_T,M3_Q,M3_T,Examen_Final
0,3.30,2.85,1.29,2.77,2.77,2.17,2.42
1,4.36,2.54,3.47,4.94,5.00,2.48,2.90
2,0.01,4.03,0.63,1.46,1.83,3.58,2.94
3,2.71,4.16,1.39,5.00,4.19,5.00,3.04
4,4.43,4.02,1.26,1.72,5.00,2.45,1.49


## 3. El "Oráculo" (Criterio Humano NUEVO)

Nuevas reglas centradas en el Examen Final:
1. **Muerte Súbita**: Si `Final < 2.5` -> **PIERDE** (sin importar nada más).
2. **Debilidad**: Si `Final < 3.0` Y `Promedio Módulos < 3.0` -> **PIERDE**.
3. **Redención**: Si `2.5 <= Final < 3.0` PERO `Todos los Módulos > 3.5` -> **GANA**.
4. En otros casos, mantenemos el criterio base (M2 es filtro, promedio simple debe aprobar).

In [178]:

def get_oracle_decisions_vectorized(weights, df):
    # Weights interpretation depends on length
    # If 3: [w_q, w_t, w_f] (Strict Notebook) -> Expand to 7
    # If 7: [m1q, m1t, m2q, m2t, m3q, m3t, final] (Standard Notebook)
    import numpy as np

    if len(weights) == 3:
        w_q, w_t, w_f = weights
        # Strict symmetry assumption
        weights_7 = np.array([w_q, w_t, w_q, w_t, w_q, w_t, w_f])
    else:
        weights_7 = weights

    # Normalize weights for module calculation (Dynamic)
    # We use the relative proportion of Q vs T in the current weights vector
    # to calculate the module grade.
    
    # Extract columns
    m1_q = df['M1_Q'].values
    m1_t = df['M1_T'].values
    m2_q = df['M2_Q'].values
    m2_t = df['M2_T'].values
    m3_q = df['M3_Q'].values
    m3_t = df['M3_T'].values
    final = df['Examen_Final'].values
    
    # Calculate Module Grades (Dynamic)
    # M1
    w1_tot = weights_7[0] + weights_7[1] + 1e-9
    n_m1 = (m1_q * weights_7[0] + m1_t * weights_7[1]) / w1_tot
    
    # M2
    w2_tot = weights_7[2] + weights_7[3] + 1e-9
    n_m2 = (m2_q * weights_7[2] + m2_t * weights_7[3]) / w2_tot
    
    # M3
    w3_tot = weights_7[4] + weights_7[5] + 1e-9
    n_m3 = (m3_q * weights_7[4] + m3_t * weights_7[5]) / w3_tot
    
    avg_modulos = (n_m1 + n_m2 + n_m3) / 3.0
    
    # Decision Logic (Vectorized)
    decisions = np.ones(len(df), dtype=int)
    
    # 1. MUERTE SÚBITA
    decisions[final < 2.5] = 0
    
    # 2. DEBILIDAD
    decisions[(final < 3.0) & (avg_modulos < 3.0)] = 0
    
    # 3. FILTRO M2 (Legacy strict rule)
    decisions[n_m2 < 2.5] = 0
    
    # 4. PROMEDIO TOTAL SIMPLE < 3.0 (Legacy rule check)
    # Note: Optimization notebooks often used simple average as baseline constraint
    # We'll calculate simple average of all raw components
    avg_total = (m1_q + m1_t + m2_q + m2_t + m3_q + m3_t + final) / 7.0
    # Actually, simpler: just check average of modules + final
    # decisions[(avg_modulos * 3 + final)/4 < 3.0] = 0
    # Let's stick to the notebook's specific legacy logic if it had one, or a general reasonable one.
    # The 'notas' notebook had: np.mean([all_cols]) < 3.0. Let's replicate roughly.
    all_raw_cols = np.stack([m1_q, m1_t, m2_q, m2_t, m3_q, m3_t, final], axis=1)
    decisions[np.mean(all_raw_cols, axis=1) < 3.0] = 0
    
    # REDEMTPION (Overrides previous fails)
    # If 2.5 <= Final < 3.0 BUT all modules > 3.5
    mask_redemption = (final >= 2.5) & (final < 3.0) & (n_m1 > 3.5) & (n_m2 > 3.5) & (n_m3 > 3.5)
    decisions[mask_redemption] = 1
    
    return decisions


## 4. Optimización Directa

Optimización con pesos restringidos (Concepto <= 15%).

In [179]:
# Inicializar columna Aprueba_Humano para referencia legacy
# Usamos pesos neutros para esta inicialización
w_init = np.ones(7)/7
df_notas['Aprueba_Humano'] = get_oracle_decisions_vectorized(w_init, df_notas)

X = df_notas[features].values
y = df_notas['Aprueba_Humano'].values


# y is no longer static, removed from assignment here or kept for reference? KEPT for reference but unused in opt.
 # Legacy static

# Pesos ahora tiene 7 valores (0..6)
# 0,1: M1 (Q,T)
# 2,3: M2 (Q,T)
# 4,5: M3 (Q,T)
# 6: Examen Final

def funcion_objetivo(weights, X, df_context):
    # 1. Oraculo Dinamico
    # Recalculamos quien aprueba segun los pesos actuales
    y_true_dynamic = get_oracle_decisions_vectorized(weights, df_context)
    
    # 2. Notas Propuestas
    notas_finales = X.dot(weights)
    passing_grade = 2.95
    
    pass_mask = (y_true_dynamic == 1)
    fail_mask = (y_true_dynamic == 0)
    
    err_pass = np.maximum(0, passing_grade - notas_finales[pass_mask])**2
    err_fail = np.maximum(0, notas_finales[fail_mask] - passing_grade)**2
    
    # Penalizacion ZONA GRIS
    gray_zone_mask = (notas_finales >= 2.9) & (notas_finales <= 3.1)
    gray_penalty = np.sum(gray_zone_mask) * 50 
    
    return np.sum(err_pass) + np.sum(err_fail) + gray_penalty

cons = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

# Quices < 30% del modulo
cons.append({'type': 'ineq', 'fun': lambda w: 0.3*w[1] - 0.7*w[0]})
cons.append({'type': 'ineq', 'fun': lambda w: 0.3*w[3] - 0.7*w[2]})
cons.append({'type': 'ineq', 'fun': lambda w: 0.3*w[5] - 0.7*w[4]})

# Examen Final >= 20% y <= 40%
cons.append({'type': 'ineq', 'fun': lambda w: w[6] - 0.20})
cons.append({'type': 'ineq', 'fun': lambda w: 0.40 - w[6]})

# Tareas <= 20% (individuales) 
cons.append({'type': 'ineq', 'fun': lambda w: 0.20 - w[1]})
cons.append({'type': 'ineq', 'fun': lambda w: 0.20 - w[3]})
cons.append({'type': 'ineq', 'fun': lambda w: 0.20 - w[5]})

# Relación Módulos: M2 = 1.1 * M1 y M1 = M3
cons.append({'type': 'eq', 'fun': lambda w: (w[2] + w[3]) - 1.1 * (w[0] + w[1])}) # M2 = 1.1 * M1
cons.append({'type': 'eq', 'fun': lambda w: (w[0] + w[1]) - (w[4] + w[5])})       # M1 = M3

bnds = [(0.01, 1.0)] * 7
w0 = np.ones(7) / 7

print("Optimizando (Dynamic Oracle)...")
# Args changed to (X, df_notas)
res = minimize(funcion_objetivo, w0, args=(X, df_notas), constraints=cons, bounds=bnds)
w_best = res.x

# MOSTRAR PESOS CLARAMENTE
pm1 = w_best[0] + w_best[1]
pm2 = w_best[2] + w_best[3]
pm3 = w_best[4] + w_best[5]
p_final = w_best[6]

print("   === PESOS ÓPTIMOS ENCONTRADOS ===")
print(f"Modulo 1 (Q+T): {pm1*100:.1f}%")
print(f"Modulo 2 (Q+T): {pm2*100:.1f}%")
print(f"Modulo 3 (Q+T): {pm3*100:.1f}%")
print(f"Examen Final  : {p_final*100:.1f}%")
print("-----------------------------------")
print(f"Suma Total: {(pm1 + pm2 + pm3 + p_final)*100:.1f}%")

# Evaluar accuracy
notas_calc = X.dot(w_best)
aprueba_model = (notas_calc >= 2.95).astype(int)
# Recalcular Oraculo Final
y_final = get_oracle_decisions_vectorized(w_best, df_notas)
acc = (y_final == aprueba_model).mean() 
print(f"   Exactitud del Modelo: {acc:.2%}")


Optimizando (Dynamic Oracle)...
   === PESOS ÓPTIMOS ENCONTRADOS ===
Modulo 1 (Q+T): 19.4%
Modulo 2 (Q+T): 21.3%
Modulo 3 (Q+T): 19.4%
Examen Final  : 40.0%
-----------------------------------
Suma Total: 100.0%
   Exactitud del Modelo: 70.95%


## 5. Tabla Detallada de Resultados (Por Módulos)

Mostramos las notas con **1 decimal** y especificamos el peso usado en el encabezado.

In [180]:
# Actualizar Oraculo con Optimos
df_notas['Aprueba_Humano'] = get_oracle_decisions_vectorized(w_best, df_notas)
df_res = df_notas.copy()
df_res['Nota_Final_Calculada'] = X.dot(w_best)
df_res['Aprueba_Modelo'] = (df_res['Nota_Final_Calculada'] >= 2.95).astype(int)

# Estados
conditions = [
    (df_res['Aprueba_Humano'] == 1) & (df_res['Aprueba_Modelo'] == 1),
    (df_res['Aprueba_Humano'] == 0) & (df_res['Aprueba_Modelo'] == 0),
    (df_res['Aprueba_Humano'] == 0) & (df_res['Aprueba_Modelo'] == 1),
    (df_res['Aprueba_Humano'] == 1) & (df_res['Aprueba_Modelo'] == 0)
]
choices = ['COINCIDE (Pasa)', 'COINCIDE (Pierde)', 'FALSO POSITIVO', 'FALSO NEGATIVO']
df_res['Estado'] = np.select(conditions, choices, default='ERROR')

cols = ['M1_Q', 'M1_T', 'M2_Q', 'M2_T', 'M3_Q', 'M3_T', 'Examen_Final', 
        'Nota_Final_Calculada', 'Aprueba_Humano', 'Estado']

sample = pd.concat([
    df_res[df_res['Estado'].str.contains('FALSO')].sample(min(5, len(df_res[df_res['Estado'].str.contains('FALSO')]))),
    df_res[df_res['Estado'].str.contains('COINCIDE')].sample(5)
])

def color_rows(s):
    if 'FALSO' in str(s['Estado']):
        return ['background-color: #ffcccc'] * len(s)
    return [''] * len(s)

print("=== REPORTE DETALLADO (ESQUEMA SIMPLE) ===")
display(sample[cols].sort_values('Estado').style.apply(color_rows, axis=1).format({
    'Nota_Final_Calculada': '{:.2f}'
}))

# --- ESTADÍSTICAS SOLICITADAS ---
total = len(df_res)
fp_count = (df_res['Estado'] == 'FALSO POSITIVO').sum()
fn_count = (df_res['Estado'] == 'FALSO NEGATIVO').sum()
tn_count = (df_res['Estado'] == 'COINCIDE (Pierde)').sum()

frac_fp = fp_count / total
frac_fn = fn_count / total

# Probabilidad de que alguien que NO debería ganar, gane (FPR = FP / (FP + TN))
total_real_negatives = fp_count + tn_count
fpr = fp_count / total_real_negatives if total_real_negatives > 0 else 0.0

print("\n=== ESTADÍSTICAS DE ERROR ===")
print(f"Fracción de Falsos Positivos (Global): {frac_fp:.2%}")
print(f"Fracción de Falsos Negativos (Global): {frac_fn:.2%}")
print(f"Prob. de ganar sin merecerlo (FPR):    {fpr:.2%}  <-- (FP / Real Negatives)")


=== REPORTE DETALLADO (ESQUEMA SIMPLE) ===


,M1_Q,M1_T,M2_Q,M2_T,M3_Q,M3_T,Examen_Final,Nota_Final_Calculada,Aprueba_Humano,Estado
1958,3.740000,4.150000,3.910000,3.460000,3.000000,2.580000,2.690000,3.15,1,COINCIDE (Pasa)
1231,2.920000,3.930000,0.730000,5.000000,1.630000,3.750000,1.560000,2.72,0,COINCIDE (Pierde)
1979,2.920000,3.530000,1.120000,1.490000,2.660000,4.090000,2.600000,2.69,0,COINCIDE (Pierde)
1730,3.490000,4.030000,0.550000,2.310000,2.670000,2.560000,1.920000,2.40,0,COINCIDE (Pierde)
390,4.000000,4.190000,1.550000,1.740000,4.700000,2.380000,2.900000,2.91,0,COINCIDE (Pierde)
442,4.480000,3.810000,0.000000,1.630000,3.160000,4.340000,5.000000,3.79,0,FALSO POSITIVO
1855,4.230000,3.670000,3.930000,3.690000,3.930000,2.700000,2.370000,3.08,0,FALSO POSITIVO
872,4.310000,4.040000,1.210000,2.190000,2.180000,3.740000,3.170000,3.10,0,FALSO POSITIVO
751,4.480000,4.030000,1.380000,2.540000,3.790000,3.590000,4.490000,3.77,0,FALSO POSITIVO
1466,4.390000,4.010000,0.620000,1.890000,3.940000,3.890000,3.090000,3.11,0,FALSO POSITIVO



=== ESTADÍSTICAS DE ERROR ===
Fracción de Falsos Positivos (Global): 29.05%
Fracción de Falsos Negativos (Global): 0.00%
Prob. de ganar sin merecerlo (FPR):    41.35%  <-- (FP / Real Negatives)
